Σε αυτό το πείραμα πειράζω τις δωρεάν άδειες ενός κλάδου και βλέπω τις αλλαγές στην τιμή της άδειας στο σ΄συστημα καθώς και τις μεταβολής των κερδών του κλάδου αυτόυ.

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import sympy_to_gurobi, Firm, Country, Sector, Regulator, get_emission
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys
from gurobipy import Model, LinExpr, QuadExpr, GRB
x = sp.symbols('x')
y = sp.symbols('y')

In [2]:

# Create Sectors and Countries
Regulator2 = Regulator('EU', permit_price= 2, emission_cap= 100)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator2)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator2)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator2)
country1 = Country('DE', 1, regulator= Regulator2)
country2 = Country('FI', 0.5, regulator= Regulator2)
country3 = Country('GR', size= 0.1, regulator= Regulator2)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 0.1*x**3 + 100/y**2, 0, 0, 0, regulator= Regulator2)
firm2 = Firm('firm2', 1, 2, x*0, 5*x**3.2 + 40/y**2, 0, 0, 0, regulator= Regulator2)
firm3 = Firm('firm3', 1, 3, x*0, x**5.2 + 30/y**2, 0, 0, 0, regulator= Regulator2)
firm4 = Firm('firm4', 2, 1, x*0, x**3.3 + 20/y**2, 0, 0, 0, regulator= Regulator2)
firm5 = Firm('firm5', 2, 2, x*0, x**2.3 + 20/y**2, 0, 0, 0, regulator= Regulator2)
firm6 = Firm('firm6', 2, 3, x*0, x**3.2 + 15/y**2, 0, 0, 0, regulator= Regulator2)
firm7 = Firm('firm7', 3, 1, x*0, x**4.1 + 40/y**2, 0, 0, 0, regulator= Regulator2)
firm8 = Firm('firm8', 3, 2, x*0, x**5.1 + 50/y**2, 0, 0, 0, regulator= Regulator2)
firm9 = Firm('firm9', 3, 3, x*0, x**5 + 60/y**2, 0, 0, 0, regulator= Regulator2)


print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(Regulator2.firm_registry)  # Dictionary of all firms

for firm in Regulator2.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)


[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0)]
[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=7, name='firm7', sector_id=3, country_id=1, actual_output=0, emission=0, profit=0)]
{1: Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), 2: Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), 3: Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), 4: Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), 5: Firm(id=5, name='firm5', sector_id=2, country_id=2, actual_out

In [3]:
Regulator2.optimize_them_all(precision = 0.1, print_diff = True, BAU = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Max diff: 0.06, permit price = 2.0, cap = 100, second_stage = False, a = 1.0000

In [4]:
for firm in Regulator2.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit)

firm1 cement DE 257.32525385607016 0.0 6273.59076591368
firm2 cement FI 257.32525385607016 0.0 6273.59076591368
firm3 cement GR 241.54945043799546 0.0 5888.97661255950
firm4 steel DE 376.13222749306016 0.0 13977.1983562917
firm5 steel FI 376.13222749306016 0.0 13977.1983562917
firm6 steel GR 376.13222749306016 0.0 13977.1983562917
firm7 paper DE 118.59141288110008 0.0 7795.70908896106
firm8 paper FI 118.55901246575188 0.0 7794.93391399515
firm9 paper GR 118.67118725249841 0.0 7803.94427937029


In [5]:
Regulator2.BAU_calculator(print_diff = True)

Max diff: 0.01, permit price = 2.0, cap = 100, second_stage = False, a = 0.73

In [6]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp3_how_Free_allocation_of_one_sector_affects_others_v0_1.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator2.emission_cap = 0.8*Regulator2.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator2.sector_registry[1].free_emission_multiplier = i
    Regulator2.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision= 1, x_high = 1000000)
    caps2.append(Regulator2.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator2.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator2.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator2.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator2.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp3_how_Free_allocation_of_one_sector_affects_others_v0_1.csv', index=False)

  0%|          | 0/93 [00:00<?, ?it/s]

Max diff: 0.11, permit price = 25.7, cap = 1792, second_stage = False, a = 1.00000000

  1%|          | 1/93 [02:08<3:17:04, 128.52s/it]

Permit price: 25.74920654296875 and total emission: 1792.2563138525213 and emission cap 1792.357702840185
Permit price: 26.226044price = 26.2, cap = 1792, second_stage = False, a = 1.00000000

  2%|▏         | 2/93 [04:12<3:11:14, 126.09s/it]

...

Permit price: 44.82269287109375 and total emission: 1792.4990144813987 and emission cap 1792.357702840185
Max diff: 0.63, permit price = 44.8, cap = 1792, second_stage = False, a = 1.00000000

 99%|█████████▉| 92/93 [2:59:54<02:03, 123.22s/it]

Permit price: 44.82269287109375 and total emission: 1792.4990144813987 and emission cap 1792.357702840185
Max diff: 0.24, permit price = 45.8, cap = 1792, second_stage = False, a = 1.00000000

100%|██████████| 93/93 [3:01:45<00:00, 117.27s/it]

Permit price: 45.7763671875 and total emission: 1789.6325598868734 and emission cap 1792.357702840185
